https://colab.research.google.com/github/google-coral/tutorials/blob/master/retrain_efficientdet_model_maker_tf2.ipynb

In [ ]:
#Not working now
#!pip install -q tflite-model-maker

In [ ]:
#https://www.reddit.com/r/tensorflow/comments/wl1pj3/colab_tflitemodelmaker_taking_ages_to_load/
!git clone https://github.com/tensorflow/examples
%cd examples/tensorflow_examples/lite/model_maker/pip_package
!pip install -e .

In [ ]:
!apt-get install libportaudio2

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Your labels map as a dictionary (zero is reserved):
label_map = {1: 'stop', 2: 'trafficlight', 3: 'crosswalk', 4: 'speedlimit50', 5: 'speedlimit60', 6: 'speedlimit70', 7: 'speedlimit80', 8: 'speedlimit90'}
#label_map = {1: 'stop' , 2: 'velocidad30', 3: 'velocidad10', 4: 'semaforoRojo', 5: 'semaforoVerde'}

train_images_dir = '/content/drive/MyDrive/tfm/signs_picar/train/images'
train_annotations_dir = '/content/drive/MyDrive/tfm/signs_picar/train/annotations'
val_images_dir = '/content/drive/MyDrive/tfm/signs_picar/validation/images'
val_annotations_dir = '/content/drive/MyDrive/tfm/signs_picar/validation/annotations'
test_images_dir = '/content/drive/MyDrive/tfm/signs_picar/test/images'
test_annotations_dir = '/content/drive/MyDrive/tfm/signs_picar/test/annotations'

#road307

In [ ]:
validation_data = object_detector.DataLoader.from_pascal_voc(val_images_dir, val_annotations_dir, label_map=label_map)

test_data = object_detector.DataLoader.from_pascal_voc(test_images_dir, test_annotations_dir, label_map=label_map)

train_data = object_detector.DataLoader.from_pascal_voc(train_images_dir, train_annotations_dir, label_map=label_map)

print(f'train count: {len(train_data)}')
print(f'validation count: {len(validation_data)}')
print(f'test count: {len(test_data)}')

In [ ]:
spec = object_detector.EfficientDetLite1Spec()

In [ ]:
model = object_detector.create(train_data=train_data, 
                               model_spec=spec, 
                               validation_data=validation_data, 
                               epochs=50, 
                               batch_size=10, 
                               train_whole_model=True)

In [ ]:
model.evaluate(test_data)

In [ ]:
TFLITE_FILENAME = 'efficientdet-lite-car_signs-eDet1.tflite'
LABELS_FILENAME = 'car_signs-labels-eDet1.txt'

model.export(export_dir='/content/drive/MyDrive/tfm/', tflite_filename=TFLITE_FILENAME, label_filename=LABELS_FILENAME,
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

In [ ]:
model.evaluate_tflite('/content/drive/MyDrive/tfm/efficientdet-lite-car_signs-eDet1.tflite', test_data)

In [ ]:
import random

images_path = test_images_dir
filenames = os.listdir(os.path.join(images_path))
random_index = random.randint(0,len(filenames)-1)
INPUT_IMAGE = os.path.join(images_path, filenames[random_index])

In [ ]:
! python3 -m pip install --extra-index-url https://google-coral.github.io/py-repo/ pycoral

In [ ]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

import tflite_runtime.interpreter as tflite 
from pycoral.adapters import common
from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file

def draw_objects(draw, objs, scale_factor, labels):
  """Draws the bounding box and label for each object."""
  COLORS = np.random.randint(0, 255, size=(len(labels), 3), dtype=np.uint8)
  for obj in objs:
    bbox = obj.bbox
    color = tuple(int(c) for c in COLORS[obj.id])
    draw.rectangle([(bbox.xmin * scale_factor, bbox.ymin * scale_factor),
                    (bbox.xmax * scale_factor, bbox.ymax * scale_factor)],
                   outline=color, width=3)
    font = ImageFont.truetype("LiberationSans-Regular.ttf", size=15)
    draw.text((bbox.xmin * scale_factor + 4, bbox.ymin * scale_factor + 4),
              '%s\n%.2f' % (labels.get(obj.id, obj.id), obj.score),
              fill=color, font=font)

# Load the TF Lite model
labels = read_label_file('/content/drive/MyDrive/tfm/car_signs-labels-eDet1.txt')
interpreter = tflite.Interpreter('/content/drive/MyDrive/tfm/efficientdet-lite-car_signs-eDet1.tflite')
interpreter.allocate_tensors()

# Resize the image for input
image = Image.open(INPUT_IMAGE)
_, scale = common.set_resized_input(
    interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))

# Run inference
interpreter.invoke()
objs = detect.get_objects(interpreter, score_threshold=0.4, image_scale=scale)

# Resize again to a reasonable size for display
display_width = 500
scale_factor = display_width / image.width
height_ratio = image.height / image.width
image = image.resize((display_width, int(display_width * height_ratio)))
draw_objects(ImageDraw.Draw(image), objs, scale_factor, labels)
image

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler

In [ ]:
NUMBER_OF_TPUS = 1
!edgetpu_compiler '/content/drive/MyDrive/tfm/efficientdet-lite-car_signs-eDet1.tflite' -d --num_segments=$NUMBER_OF_TPUS